In [14]:
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch



def tokenization(inputpath):

    df = pd.read_csv(inputpath)

    df['sentence'] = df['sentence'].astype(str)

    tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    model = AutoModel.from_pretrained("yiyanghkust/finbert-tone")

    tokens = tokenizer(df['sentence'].tolist(), padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1) 


    return embeddings




In [15]:
import csv
import re

def txt_to_csv(input_txt_path, output_csv_path):
    with open(input_txt_path, 'r') as file:
        text = file.read()
    
    # Split text by sentences
    sentences = re.split(r'(?<=[.!?]) +', text)
    
    # Remove commas from sentences
    sentences = [sentence.replace(',', '') for sentence in sentences]
    
    # Write sentences to CSV
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['sentence'])
        for sentence in sentences:
            writer.writerow([sentence])

txt_to_csv('text.txt', 'output.csv')

In [ ]:
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import nltk
from nltk.corpus import stopwords


def cleaning(inputpath, outputpath):

    df = pd.read_csv(inputpath)

    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', str(x)) if pd.notnull(x) else '')
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    stop_words = set(stopwords.words('english'))
    df = df.applymap(lambda x: ' '.join([word for word in x.split() if word not in stop_words]) if isinstance(x, str) else x)


    tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    model = AutoModel.from_pretrained("yiyanghkust/finbert-tone")
    tokens = tokenizer(df['sentence'].tolist(), padding=True, truncation=True, return_tensors="pt")
    df.to_csv(outputpath, index=False)

cleaning('output.csv', 'output_cleaned.csv')

/var/folders/34/4m3rx1zd32dgvl3g8t313g900000gn/T/ipykernel_46791/3544532574.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/var/folders/34/4m3rx1zd32dgvl3g8t313g900000gn/T/ipykernel_46791/3544532574.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ' '.join([word for word in x.split() if word not in stop_words]) if isinstance(x, str) else x)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
import torch
class ANNs(torch.nn.Module):
    def __init__(self):
        super(ANNs,self).__init__()
        self.fc2 = torch.nn.Linear(768, 512)
        self.fc3 = torch.nn.Linear(512, 256)
        self.fc4 = torch.nn.Linear(256, 2)

    def forward(self, x):
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)

        return x
    

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

network = ANNs()

network.load_state_dict(torch.load('../nlpmodel/best_model.pth'))

network.eval()

embeddings = tokenization('output.csv')

with torch.no_grad():
    predictions = network(embeddings).argmax(dim=1).cpu().numpy()
    print(predictions)


/var/folders/34/4m3rx1zd32dgvl3g8t313g900000gn/T/ipykernel_46791/2666761575.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  network.load_state_dict(torch.load('../nlpmod

[1 1 1 1 1 0 1 0]
